In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interpolate

from DarkCapPy import *
import DarkCapPy.DarkPhoton as DP

import matplotlib.pylab as plt
import seaborn as sns



In [ ]:
#infilename = 'rates_muons_alpha_therm_one_month.parquet'
infilename = 'rates_muons_alpha_max_one_month.parquet'

df = pd.read_parquet(infilename)
df

In [ ]:
filter = df['rate_CMS'] > 50

df[filter]

In [ ]:
df.plot.scatter(x='mx', y='rate_CMS')

In [ ]:
df_max_rates = None
for mx_test in np.arange(1000, 11000, 1000):
    print(mx_test)
    filter = (df['mx']==mx_test) & (df['rate_CMS'] > 10)
    
    high_rate = df[filter]['rate_CMS'].max()
    print(high_rate)

    filter = filter & (df['rate_CMS']==high_rate)

    if df is None:
        df_max_rates = df[filter].copy()
    else:
        df_max_rates = pd.concat([df_max_rates, df[filter].copy()])

In [ ]:
df_max_rates

In [ ]:
df_max_rates.to_csv('maximum_rates_at_CMS.csv')